In [111]:
import os
import pandas as pd
import requests
from bs4 import BeautifulSoup as bs

pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', -1)
pd.set_option('display.max_columns', None)

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [112]:
dir_item = '/home/jake/Documents/News_Item/'
dir_proj = dir_item + 'uniqlo/'
dir_tableau = dir_proj + 'tableau_data/'

if not os.path.exists(dir_tableau):
    os.makedirs(dir_tableau)

# get_json()

In [52]:
a = [1,2,3]
a.extend([4,5,6])
a

[1, 2, 3, 4, 5, 6]

In [96]:
def get_json(country_code):
    '''
    offset starts from 0
    '''
    import math
    offset = 0
    def get_url(offset):
        return f'https://store.fastretailing.com/api/{country_code}/uniqlo/200/getStoreList.json?limit=100&offset={offset}&lang=english&r=storelocator&RESET=true'
    url = get_url(offset)
    print(url)
    r = requests.get(url)
    jsn = r.json()
    stores = jsn.get('result', {}).get('stores', [])
    total_count = jsn.get('result', {}).get('total_count', 0)
    # if total_count == 192: 0, 1 => loop_count = 1
    loop_count = math.floor(total_count / 100)
    for i in range(loop_count):
        offset += 100
        url = get_url(offset)
        print(url)
        r = requests.get(url)
        tmp = r.json()
        tmp_stores = tmp.get('result', {}).get('stores', [])
        stores.extend(tmp_stores)
    return jsn


code_ca = 'kr'
jsn = get_json(code_ca)
kr = get_df_from_json(jsn)
kr.shape

https://store.fastretailing.com/api/kr/uniqlo/200/getStoreList.json?limit=100&offset=0&lang=english&r=storelocator&RESET=true
https://store.fastretailing.com/api/kr/uniqlo/200/getStoreList.json?limit=100&offset=100&lang=english&r=storelocator&RESET=true


(192, 63)

# Code

In [98]:
countries = ['KOREA', 'AUSTRALIA', 'CANADA', 'CHINA', 'DENMARK', 'FRANCE', 'GERMANY', 'HONG KONG', 'INDIA', 'INDONESIA', 'ITALY',
'JAPAN', 'MALAYSIA', 'PHILIPPINES', 'RUSSIA', 'SINGAPORE', 'SPAIN', 'SWEDEN', 'TAIWAN', 'THAILAND', 'UK', 'USA']
codes = ['kr', 'au', 'ca', 'cn', 'dk', 'fr', 'de', 'hk', 'in', 'id', 'it', 'jp', 'my', 'ph', 'ru', 'sg', 'es', 'se', 'tw', 'th', 'uk', 'us']
jsons = []

for code in codes:
    jsons.append(get_json(code))

https://store.fastretailing.com/api/kr/uniqlo/200/getStoreList.json?limit=100&offset=0&lang=english&r=storelocator&RESET=true
https://store.fastretailing.com/api/kr/uniqlo/200/getStoreList.json?limit=100&offset=100&lang=english&r=storelocator&RESET=true
https://store.fastretailing.com/api/au/uniqlo/200/getStoreList.json?limit=100&offset=0&lang=english&r=storelocator&RESET=true
https://store.fastretailing.com/api/ca/uniqlo/200/getStoreList.json?limit=100&offset=0&lang=english&r=storelocator&RESET=true
https://store.fastretailing.com/api/cn/uniqlo/200/getStoreList.json?limit=100&offset=0&lang=english&r=storelocator&RESET=true
https://store.fastretailing.com/api/cn/uniqlo/200/getStoreList.json?limit=100&offset=100&lang=english&r=storelocator&RESET=true
https://store.fastretailing.com/api/cn/uniqlo/200/getStoreList.json?limit=100&offset=200&lang=english&r=storelocator&RESET=true
https://store.fastretailing.com/api/cn/uniqlo/200/getStoreList.json?limit=100&offset=300&lang=english&r=storeloc

In [99]:
len(jsons)

22

In [100]:
counts = [j['result']['total_count'] for j in jsons]
counts

[192,
 20,
 11,
 751,
 1,
 26,
 9,
 29,
 0,
 26,
 0,
 827,
 50,
 58,
 37,
 27,
 2,
 1,
 68,
 50,
 13,
 52]

In [101]:
192 / sum(counts) * 100

8.533333333333333

In [102]:
sum(counts)

2250

In [115]:
data = list(zip(countries, counts))
ranking = pd.DataFrame(data)
ranking.columns = ['나라', '매장 수']
ranking = ranking.sort_values('매장 수', ascending=False)
ranking

,나라,매장 수
11,JAPAN,827
3,CHINA,751
0,KOREA,192
18,TAIWAN,68
13,PHILIPPINES,58
21,USA,52
19,THAILAND,50
12,MALAYSIA,50
14,RUSSIA,37
7,HONG KONG,29


# Export: ranking

In [116]:
# ranking.to_csv(dir_tableau + 'ranking.tsv', sep='\t', index=False)

In [118]:
def get_df_from_json(json):
    stores = json['result']['stores']
    return pd.DataFrame(stores)


def get_df_from_jsons(jsons, idx=0):
    return get_df_from_json(jsons[0])


kr = get_df_from_jsons(jsons, 0)
kr.head()

,address,address_phonogram,announcement,area0_code,area0_name,area1_code,area1_name,baby_flag,building,category_label,click_and_collect_flg,country_name_short,distance,floor_map_flg,fri_close_at,fri_open_at,g1_ims_store_id_13,g1_ims_store_id_4,g1_ims_store_id_6,hol_close_at,hol_open_at,inventory_flg,kids_flag,lat,lon,maternity_flg,men_flg,mon_close_at,mon_open_at,municipality,my_store_flag,number,open_date,open_hours,parking_flg,phone,postcode,recruit_flg,recruit_link_url,regional_recruit_flg,regional_recruit_link_url,sat_close_at,sat_open_at,store_delivery_flg,store_holiday,store_id,store_images,store_name,store_type_code,store_url,sun_close_at,sun_open_at,thu_close_at,thu_open_at,tue_close_at,tue_open_at,wd_close_at,wd_open_at,we_close_at,we_open_at,wed_close_at,wed_open_at,women_flg
0,,,,0,Others,1,서울특별시,0,,[],0,KR,,0,21:30,11:30,S1004000001,1,101465,,,1,0,37.5635033,126.9817548,0,0,21:30,11:30,,0,,,,1,02-2118-5371,100729,0,,0,,21:30,11:30,0,,10500001,"[{'url': 'http://store.fastretailing.com/assets/img/uniqlo/data/default.png', 'width': 640, 'height': 417}]",,1000,,21:30,11:30,21:30,11:30,21:30,11:30,,,,,21:30,11:30,0
1,,,,0,Others,7,대구광역시,1,,"[{'name': 'BABY'}, {'name': 'KIDS'}]",0,KR,,0,20:30,10:30,,,112028,,,1,1,35.8737382,128.6032642,0,0,20:00,10:30,,0,,,,1,053-660-3880,,0,,0,,20:30,10:30,0,,10500023,"[{'url': 'http://store.fastretailing.com/assets/img/uniqlo/data/default.png', 'width': 640, 'height': 417}]",,1000,,20:30,10:30,20:00,10:30,20:00,10:30,,,,,20:00,10:30,0
2,,,,0,Others,14,전라북도,1,,"[{'name': 'BABY'}, {'name': 'KIDS'}]",0,KR,,0,20:30,10:30,S1004000025,25,110609,,,1,1,35.8344854,127.1219267,0,0,20:00,10:30,,0,,,,1,063-289-3927,560734,0,,0,,20:30,10:30,0,,10500025,"[{'url': 'http://store.fastretailing.com/assets/img/uniqlo/data/default.png', 'width': 640, 'height': 417}]",,1000,,20:30,10:30,20:00,10:30,20:00,10:30,,,,,20:00,10:30,0
3,,,,0,Others,3,경기도,1,,"[{'name': 'BABY'}, {'name': 'KIDS'}, {'name': '임부복'}]",0,KR,,0,21:00,10:30,S1004000346,346,113206,,,1,1,37.24225260000001,127.40007979999996,1,0,20:30,10:30,,0,,,,1,031-887-4101,,0,,0,,21:00,10:30,0,,10500346,"[{'url': 'http://store.fastretailing.com/assets/img/uniqlo/data/default.png', 'width': 640, 'height': 417}]",,1000,,21:00,10:30,20:30,10:30,20:30,10:30,,,,,20:30,10:30,0
4,,,,0,Others,3,경기도,1,,"[{'name': 'BABY'}, {'name': 'KIDS'}]",0,KR,,0,22:00,10:00,S1004000415,415,110004,,,1,1,37.3171241,126.8479131,0,0,22:00,10:00,,0,,,,1,031-487-8442,426-837,0,,0,,22:00,10:00,0,,10500415,"[{'url': 'http://store.fastretailing.com/assets/img/uniqlo/data/default.png', 'width': 640, 'height': 417}]",,1000,,22:00,10:00,22:00,10:00,22:00,10:00,,,,,22:00,10:00,0


In [117]:
cols = ['country_name_short', 'lat', 'lon', 'area1_name']

In [121]:
stores = pd.DataFrame()
for jsn in jsons:
    tmp = get_df_from_json(jsn)
    stores = stores.append(tmp)

stores = stores[cols]
print(stores.shape)
stores.head()

(2250, 4)


,country_name_short,lat,lon,area1_name
0,KR,37.5635033,126.9817548,서울특별시
1,KR,35.8737382,128.6032642,대구광역시
2,KR,35.8344854,127.1219267,전라북도
3,KR,37.24225260000001,127.40007979999996,경기도
4,KR,37.3171241,126.8479131,경기도


# Export: stores

In [125]:
# stores.to_csv(dir_tableau + 'store_location.tsv', sep='\t', index=False)

In [105]:
kr['store_id'].nunique()

192

In [107]:
kr.shape

(192, 63)

In [108]:
kr['area1_name'].value_counts()

서울특별시      53
경기도        46
부산광역시      13
대구광역시      12
경상남도       11
인천광역시      9 
대전광역시      7 
전라북도       6 
광주광역시      6 
충청남도       6 
울산광역시      5 
충청북도       4 
경상북도       4 
전라남도       4 
강원도        3 
제주특별자치도    2 
세종특별자치시    1 
Name: area1_name, dtype: int64